<a href="https://colab.research.google.com/github/soheilpaper/-tft-2.4-ili9341-STM32/blob/master/Copy_of_COLAB_stock_prediction_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adapted by Carlos Toxtli http://www.carlostoxtli.com/#colab-stock-1

Credits: https://github.com/Kulbear/stock-prediction

In [11]:
%cd /content/
!git clone https://github.com/Kulbear/stock-prediction
%cd stock-prediction

/content
fatal: destination path 'stock-prediction' already exists and is not an empty directory.
/content/stock-prediction


# Stock Prediction with Recurrent Neural Network

Deep learning is involved a lot in the modern quantitive financial field. There are many different neural networks can be applied to stock price prediction problems. The recurrent neural network, to be specific, the Long Short Term Memory(LSTM) network outperforms others architecture since it can take advantage of predicting time series (or sequentially) involved result with a specific configuration.

We will make a really simple LSTM with Keras to predict the stock price in the Chinese stock.

In [12]:
import time
import math
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
import numpy as np
import pandas as pd
import sklearn.preprocessing as prep

## Import Data

I have introduced how we get this dataset both in README.md and in another notebook.

In [13]:
#from google.colab import drive
#drive.mount('/content/gdrive')
#!cp '/content/gdrive/MyDrive/Amir Farzin Project/so-so.csv' /content/stock-prediction
!wget https://transfer.sh/o5273/so-so.csv

--2021-03-10 08:53:17--  https://transfer.sh/o5273/so-so.csv
Resolving transfer.sh (transfer.sh)... 144.76.136.153
Connecting to transfer.sh (transfer.sh)|144.76.136.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32767757 (31M) [text/csv]
Saving to: ‘so-so.csv.1’

so-so.csv.1         100%[===================>]  31.25M  12.1MB/s    in 2.6s    

2021-03-10 08:53:22 (12.1 MB/s) - ‘so-so.csv.1’ saved [32767757/32767757]



In [14]:
#!curl  --upload-file /content/stock-prediction/so-so.csv  https://transfer.sh/so-so.csv

https://transfer.sh/XoueU/so-so.csv

In [15]:
# column_values_df = df["time"].unique()[0:20]
# print (column_values_df[0:20])

In [16]:
df = pd.read_csv('so-so.csv')
df.head()

,time,-1-,-2-,-3-,-4-,-5-,out
0,25:32.2,-8.0,0.018727,1.334451,60.080002,37.026861,15147.67255
1,25:32.3,-8.0,0.018305,1.344409,60.080002,37.011600,15027.05899
2,25:32.4,-8.0,0.013568,1.322260,59.820000,37.011600,15270.12537
3,25:32.5,-8.0,0.015240,1.330974,59.820000,37.011600,15162.10324
4,25:32.6,-8.0,0.014344,1.329409,59.820000,37.011600,15173.13050


In [17]:
df.head()

,time,-1-,-2-,-3-,-4-,-5-,out
0,25:32.2,-8.0,0.018727,1.334451,60.080002,37.026861,15147.67255
1,25:32.3,-8.0,0.018305,1.344409,60.080002,37.011600,15027.05899
2,25:32.4,-8.0,0.013568,1.322260,59.820000,37.011600,15270.12537
3,25:32.5,-8.0,0.015240,1.330974,59.820000,37.011600,15162.10324
4,25:32.6,-8.0,0.014344,1.329409,59.820000,37.011600,15173.13050


In [18]:
import re, regex
ddd=df['time'].str
df['day_of_week']= ddd.split('.').str[-1]
df['time']= ddd.split('.').str[0]
df['day_of_week']=pd.to_datetime(df['day_of_week'], format='%H:%M', errors='ignore') #
# df=df[['time','day_of_week','-1-','-2-','-3-','-4-','-5-','out']]
df=df[['-1-','-2-','-3-','-4-','-5-','out']]
df.head()



,-1-,-2-,-3-,-4-,-5-,out
0,-8.0,0.018727,1.334451,60.080002,37.026861,15147.67255
1,-8.0,0.018305,1.344409,60.080002,37.011600,15027.05899
2,-8.0,0.013568,1.322260,59.820000,37.011600,15270.12537
3,-8.0,0.015240,1.330974,59.820000,37.011600,15162.10324
4,-8.0,0.014344,1.329409,59.820000,37.011600,15173.13050


In [19]:
df.columns 
df.dtypes

-1-    float64
-2-    float64
-3-    float64
-4-    float64
-5-    float64
out    float64
dtype: object

In [20]:
# # head, sep, tail = df['time'].partition('.')
# df['day_of_week']= df['time'].str.split('.')
# dd=df['day_of_week']

# for index, row in dd:
#   print (index)

# print(df['day_of_week'])
# # column_values_df = df["day_of_week"].unique()[0:20]
# # print (column_values_df[0:20])

# # df['time']=df['time'].str.split('.', 1))
# # df['time']=pd.to_datetime(df['time'].str.replace('.',':'), format='%H:%M', errors='ignore')

# # # df['time']=pd.to_datetime(df['time'], format='%H:%M', errors='ignore').df.dayofweek #
# # # pd.to_datetime(df['time']).dt.date
# # # pd.to_timedelta(df['time'])

# # df.head()
# # df.columns 
# # df.dtypes
# # # pd.to_datetime(df['time'])

In [21]:
# df['time']=pd.to_datetime(df['time'], format='%M:%S') #, errors='ignore'
# df['day_of_week']=pd.to_numeric(df['day_of_week'], errors='coerce')
# pd.to_datetime(df['time']).dt.date
# pd.to_timedelta(df['time'])
df.head()
df.columns 
df.dtypes
# pd.to_datetime(df['time'])

-1-    float64
-2-    float64
-3-    float64
-4-    float64
-5-    float64
out    float64
dtype: object

In [22]:
# df['time']=df['time'].values.astype('float32')
# df['day_of_week']=df['day_of_week'].values.astype('float32')
# df=df[['time','day_of_week','-1-','-2-','-3-','-4-','-5-','out']]
df=df[['-1-','-2-','-3-','-4-','-5-','out']]




In [23]:
df.head()

,-1-,-2-,-3-,-4-,-5-,out
0,-8.0,0.018727,1.334451,60.080002,37.026861,15147.67255
1,-8.0,0.018305,1.344409,60.080002,37.011600,15027.05899
2,-8.0,0.013568,1.322260,59.820000,37.011600,15270.12537
3,-8.0,0.015240,1.330974,59.820000,37.011600,15162.10324
4,-8.0,0.014344,1.329409,59.820000,37.011600,15173.13050


In [24]:
print(df[:: -1])

              -1-       -2-       -3-        -4-        -5-           out
407252 -12.480979  0.490826  6.650262  47.619999  23.107448   9657.000000
407251 -12.473625  0.487278  6.642516  47.619999  23.107448   9667.520491
407250 -12.471359  0.482807  6.648013  47.619999  23.107448   9657.875004
407249 -12.470004  0.482281  6.656000  47.619999  23.107448   9644.081021
407248 -12.466690  0.484988  6.653695  47.619999  23.107448   9645.691192
...           ...       ...       ...        ...        ...           ...
4       -8.000000  0.014344  1.329409  59.820000  37.011600  15173.130500
3       -8.000000  0.015240  1.330974  59.820000  37.011600  15162.103240
2       -8.000000  0.013568  1.322260  59.820000  37.011600  15270.125370
1       -8.000000  0.018305  1.344409  60.080002  37.011600  15027.058990
0       -8.000000  0.018727  1.334451  60.080002  37.026861  15147.672550

[407253 rows x 6 columns]


## Preprocess Data

We can see from the table above that the scales of our features vary a lot. It will be beneficial to normalize your training data before you feed data into your model. "Having different features with widely different scales fed to your model will cause the network to weight the features not equally. This can cause a falsely prioritisation of some features over the others in the representation." Read [here](http://stackoverflow.com/a/43526354/6670143) for more detailed explanation.

In [25]:
def standard_scaler(X_train, X_test):
    train_samples, train_nx, train_ny = X_train.shape
    test_samples, test_nx, test_ny = X_test.shape
    
    X_train = X_train.reshape((train_samples, train_nx * train_ny))
    X_test = X_test.reshape((test_samples, test_nx * test_ny))
    
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    
    X_train = X_train.reshape((train_samples, train_nx, train_ny))
    X_test = X_test.reshape((test_samples, test_nx, test_ny))
    
    return X_train, X_test

Split the data to X_train, y_train, X_test, y_test

In [26]:
def preprocess_data(stock, seq_len):
    amount_of_features = len(stock.columns)
    data = stock.values
    
    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index : index + sequence_length])
        
    result = np.array(result)
    row = round(0.9 * result.shape[0])
    train = result[: int(row), :]
    
    train, result = standard_scaler(train, result)
    
    X_train = train[:, : -1]
    y_train = train[:, -1][: ,-1]
    X_test = result[int(row) :, : -1]
    y_test = result[int(row) :, -1][ : ,-1]
 
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], amount_of_features))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], amount_of_features))  
 
    return [X_train, y_train, X_test, y_test]

## Build the LSTM Network

Here we will build a simple RNN with 2 LSTM layers.
The architecture is:
    
    LSTM --> Dropout --> LSTM --> Dropout --> Fully-Conneted(Dense)

In [27]:
def build_model(layers):
    model = Sequential()
 
    # By setting return_sequences to True we are able to stack another LSTM layer
    model.add(LSTM(
        input_dim=layers[0],
        units=layers[1],
        return_sequences=True))
    model.add(Dropout(0.4))
 
    model.add(LSTM(
        layers[2],
        return_sequences=False))
    model.add(Dropout(0.3))
 
    model.add(Dense(
        units=layers[3]))
    model.add(Activation("linear"))
 
    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop", metrics=['accuracy'])
    print("Compilation Time : ", time.time() - start)
    model.summary()
    return model

In [28]:
window = 20
X_train, y_train, X_test, y_test = preprocess_data(df[:: -1], window)
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (366509, 20, 6)
y_train (366509,)
X_test (40723, 20, 6)
y_test (40723,)


In [29]:
model = build_model([X_train.shape[2], window, 100, 1])

Compilation Time :  0.010120153427124023
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 20)          2160      
_________________________________________________________________
dropout (Dropout)            (None, None, 20)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               48400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 50,661
Trainable params: 50,661
Non-trainable params: 0
___________

## Training the Network

You can play with the hyperparameters here. The training process can take a significant time if you are using a average GPU or CPU(worse!).

In [ ]:
model.fit(
    X_train,
    y_train,
    batch_size=768,
    epochs=5,
    validation_split=0.1,
    verbose=0)

In [ ]:
trainScore = model.evaluate(X_train, y_train, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))

testScore = model.evaluate(X_test, y_test, verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore[0], math.sqrt(testScore[0])))

## Visualize the Prediction

In [ ]:
diff = []
ratio = []
pred = model.predict(X_test)
for u in range(len(y_test)):
    pr = pred[u][0]
    ratio.append((y_test[u] / pr) - 1)
    diff.append(abs(y_test[u] - pr))

In [ ]:
# TODO: Scale it back

In [ ]:
# if using within a jupyter notebook
%matplotlib inline 

import matplotlib
import matplotlib.pyplot as plt2

plt2.plot(pred, color='red', label='Prediction')
plt2.plot(y_test, color='blue', label='Ground Truth')
plt2.legend(loc='upper left')
plt2.show()